In [1]:
from pathlib import Path
from time import strftime

def get_run_logidir(root_logdir="my_logs"):
    return Path(root_logdir)/strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logidir()
run_logdir

WindowsPath('my_logs/run_2024_02_01_12_11_35')

In [2]:
import tensorflow as tf

class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation='relu', **kwargs):
        super().__init__(**kwargs) # needed to support nameing the model
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)
        
    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output
    
model = WideAndDeepModel(30, activation="relu", name="my_cool_model")

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

housing = pd.read_csv('data/housing.csv')
housing.dropna(inplace=True)
housing.drop(columns=['ocean_proximity'], inplace=True)

X = housing.drop(columns=['median_house_value'])
y = housing['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train_wide, X_train_deep = X_train.iloc[:, :5], X_train.iloc[:, 2:]
X_test_wide, X_test_deep = X_test.iloc[:, :5], X_test.iloc[:, 2:]
X_new_wide, X_new_deep = X_test_wide.iloc[:3], X_test_deep.iloc[:3]

In [4]:
model.compile(
    loss = ("mse","mse"),
    loss_weights= (0.9, 0.1),
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

tensorboard_callbacks = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))
history = model.fit((X_train_wide, X_train_deep), (y_train,y_train), epochs=10, validation_split=0.1, callbacks=tensorboard_callbacks)

Epoch 1/10
489/489 [==============================] - 4s 5ms/step - loss: 38959165440.0000 - output_1_loss: 38647672832.0000 - output_2_loss: 41762496512.0000 - output_1_root_mean_squared_error: 196590.1094 - output_2_root_mean_squared_error: 204358.7500 - val_loss: 28005935104.0000 - val_output_1_loss: 28003168256.0000 - val_output_2_loss: 28030822400.0000 - val_output_1_root_mean_squared_error: 167341.4688 - val_output_2_root_mean_squared_error: 167424.0781
Epoch 2/10
489/489 [==============================] - 2s 4ms/step - loss: 27424192512.0000 - output_1_loss: 27414765568.0000 - output_2_loss: 27509090304.0000 - output_1_root_mean_squared_error: 165574.0469 - output_2_root_mean_squared_error: 165858.6406 - val_loss: 27540916224.0000 - val_output_1_loss: 27537000448.0000 - val_output_2_loss: 27576121344.0000 - val_output_1_root_mean_squared_error: 165942.7656 - val_output_2_root_mean_squared_error: 166060.5938
Epoch 3/10
489/489 [==============================] - 2s 4ms/step - loss

In [5]:
# changing the learning rate
model.compile(
    loss = ("mse","mse"),
    loss_weights= (0.9, 0.1),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.003),
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

tensorboard_callbacks = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200), update_freq='epoch')
history = model.fit((X_train_wide, X_train_deep), (y_train,y_train), epochs=10, validation_split=0.1, callbacks=tensorboard_callbacks)

Epoch 1/10
489/489 [==============================] - 4s 5ms/step - loss: 14007263232.0000 - output_1_loss: 13990251520.0000 - output_2_loss: 14160351232.0000 - output_1_root_mean_squared_error: 118280.3906 - output_2_root_mean_squared_error: 118997.2734 - val_loss: 11404414976.0000 - val_output_1_loss: 11397751808.0000 - val_output_2_loss: 11464416256.0000 - val_output_1_root_mean_squared_error: 106760.2500 - val_output_2_root_mean_squared_error: 107072.0156
Epoch 2/10
489/489 [==============================] - 2s 4ms/step - loss: 11396670464.0000 - output_1_loss: 11393667072.0000 - output_2_loss: 11423703040.0000 - output_1_root_mean_squared_error: 106741.1250 - output_2_root_mean_squared_error: 106881.7266 - val_loss: 10783642624.0000 - val_output_1_loss: 10780917760.0000 - val_output_2_loss: 10808176640.0000 - val_output_1_root_mean_squared_error: 103831.1953 - val_output_2_root_mean_squared_error: 103962.3828
Epoch 3/10
489/489 [==============================] - 2s 4ms/step - loss